In [ ]:
import torch
from utils.fda import FDA_source_to_target
from utils.aug import ExtCompose, ExtToTensor, ExtRandomHorizontalFlip , ExtScale , ExtRandomCrop, ExtGaussianBlur, ExtColorJitter, ExtRandomCompose, ExtResize, ExtNormalize
from datasets.cityscapes import CityScapes
from datasets.GTA5 import GTA5
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np

## FDA TEST

In [ ]:
beta = 0.05
h = 512
w = 1024
b = (  np.floor(np.amin((h,w))*beta)  ).astype(int)
b

In [ ]:
standard_transformations = ExtCompose([ExtScale(0.5,interpolation=Image.Resampling.BILINEAR), ExtToTensor()])
transformations = ExtCompose([ExtResize((512,1024)), ExtToTensor()])
target_transformations = standard_transformations

train_dataset = GTA5(root='dataset',transforms=transformations)
target_dataset_train = CityScapes(split='train',transforms=target_transformations)

source_dataloader_train = DataLoader(train_dataset,
                batch_size=1,
                shuffle=False,
                num_workers=1,
                pin_memory=False,
                drop_last=True)
target_dataloader_train = DataLoader(target_dataset_train,
                batch_size=1,
                shuffle=False,
                num_workers=1,
                pin_memory=False,
                drop_last=True)

for i, (source_data, target_data) in enumerate(zip(source_dataloader_train,target_dataloader_train)):
  
    if i > 10:
        break

    source_data,source_label = source_data
    target_data,_ = target_data
    source_data = source_data.cuda()
    source_label = source_label.long().cuda()
    target_data = target_data.cuda()

    t_source_data = FDA_source_to_target(source_data, target_data, beta=beta)

    # Visualize the image
    #Image.fromarray(target_data.detach().cpu().numpy().astype('uint8')[0].transpose(1, 2, 0)).show()
    #Image.fromarray(source_data.detach().cpu().numpy().astype('uint8')[0].transpose(1, 2, 0)).show()
    Image.fromarray(t_source_data.detach().cpu().numpy().astype('uint8')[0].transpose(1, 2, 0)).show()

In [ ]:
GTA_MEAN = torch.tensor([105.8735, 109.0072, 101.7702])
GTA_STD = torch.tensor([56.2629, 55.8382, 55.4247])

In [ ]:
standard_transformations = ExtCompose([ExtResize((512,1024)), ExtToTensor()])
transformations = ExtCompose([ExtResize((512,1024)), ExtToTensor(), ExtNormalize(mean=GTA_MEAN,std=GTA_STD)])
target_transformations = standard_transformations

train_dataset = GTA5(root='dataset',transforms=transformations)
target_dataset_train = CityScapes(split='train',transforms=transformations)

source_dataloader_train = DataLoader(train_dataset,
                batch_size=1,
                shuffle=False,
                num_workers=6,
                pin_memory=False,
                drop_last=True)
target_dataloader_train = DataLoader(target_dataset_train,
                batch_size=1,
                shuffle=False,
                num_workers=6,
                pin_memory=False,
                drop_last=True)
mean = 0.
std = 0.
K = 1000
print(len(source_dataloader_train))
for i , (data , labels) in enumerate(source_dataloader_train):
    mean += data.mean(keepdim=True,dim=(0,2,3))
    std += data.std(keepdim=True, dim=(0,2,3))
    if i % 100 == 0:
        print(i)
for i , (data , labels) in enumerate(target_dataloader_train):
    mean += data.mean(keepdim=True,dim=(0,2,3))
    std += data.std(keepdim=True, dim=(0,2,3))
    if i % 100 == 0:
        print(i)
mean /= (len(source_dataloader_train) + len(target_dataloader_train))
std /= (len(source_dataloader_train) + len(target_dataloader_train))
print(mean.view(-1))
print(std.view(-1))

In [19]:
from utils.aug import ExtToV2Tensor, V2Normalize, ExtCompose, ExtResize
from datasets.cityscapes import CityScapes
from datasets.GTA5 import GTA5
from torch.utils.data import DataLoader
from PIL import Image
from torchvision.transforms import v2
standard_transformations = ExtCompose([ExtResize((512,1024)), ExtToV2Tensor(), V2Normalize(scale=True)])
train_dataset = GTA5(root='dataset',transforms=standard_transformations)
mean=[-0.485, -0.456, -0.406]
std=[1/0.229, 1/0.224, 1/0.225]
source_dataloader_train = DataLoader(train_dataset,
                batch_size=1,
                shuffle=False,
                num_workers=0,
                pin_memory=False,
                drop_last=True)

for i , (data,label) in enumerate(source_dataloader_train):
    if i > 0:
        break
    
    data, _ = V2Normalize(mean=mean,std=std)(data,_)
    #data *= -255
    print(data[0])
    Image.fromarray(data[0].detach().cpu().numpy().astype('uint8').transpose(1, 2, 0)).show()


tensor([[[522.5983, 518.2314, 492.0306,  ..., 369.7599, 369.7599, 369.7599],
         [522.5983, 518.2314, 496.3974,  ..., 369.7599, 374.1266, 374.1266],
         [522.5983, 522.5983, 513.8647,  ..., 374.1266, 378.4935, 378.4935],
         ...,
         [330.4585, 334.8253, 339.1922,  ..., 260.5895, 269.3231, 269.3231],
         [352.2926, 356.6594, 352.2926,  ..., 264.9563, 225.6550, 216.9214],
         [361.0262, 365.3930, 356.6594,  ..., 238.7555, 221.2882, 212.5546]],

        [[501.2500, 501.2500, 474.4643,  ..., 336.0714, 336.0714, 336.0714],
         [501.2500, 501.2500, 483.3929,  ..., 340.5357, 340.5357, 345.0000],
         [501.2500, 501.2500, 496.7857,  ..., 345.0000, 349.4642, 349.4642],
         ...,
         [295.8928, 295.8928, 300.3571,  ..., 309.2857, 318.2143, 313.7500],
         [313.7500, 318.2143, 313.7500,  ..., 313.7500, 291.4286, 286.9643],
         [322.6785, 327.1429, 318.2143,  ..., 300.3571, 286.9643, 286.9643]],

        [[442.3555, 442.3555, 424.5778,  ...